# ÁLGEBRA LINEAR COMPUTACIONAL

## SEGUNDA AVALIAÇÃO - 2020.2 - PROFESSOR ERITO MARQUES   

---
### GRUPO:
**Ícaro Santos Barcelos Pereira - _2017780187_**\
**Leandro Bataglia Pereira - _2018780199_**\
**Pedro Henrique Farolfi Camelo - _2018780326_**

---

Repositório dos algoritmos solicitados na segunda prova da disciplina de Álgebra Linear Computacional.
##### Instruções:
As respostas devem ser enviadas no formato jupyter notebook.\
Os trabalhos devem ser feitos em grupo. Não é permitida troca de informações entre os grupos.\
Fixada a data de entrega, não serão aceitos trabalhos entregues fora dessa data.

In [ ]:
import numpy as np
import pandas as pd
import normas
from cholesky import decomposicao_cholesky
from gauss import eliminacao_gauss
from fatoracaoLU import fatoracaoLU
from substituicao import resolve_substituicao
from sympy import Matrix, init_printing
init_printing()
#BIBLIOTECA QUE NÃO ESTOU USANDO MAS PODE SER ÚTIL PARA EXIBIR AS MATRIZES/VETORES = display(Matrix(*variavel que representa o dado*)

In [ ]:
# Coverti isso aqui em um procedimento, a parada é que vc pode mandar isso
# ocorrer no início de qualquer célula e pode alterar o excel em tempo de execução :)

# CONVERSÃO DOS DADOS EM ARRAYS DA NUMPY COM TYPE 'float64'
def ler():
    df_matriz = pd.read_excel(r"dados/matriz_A.xlsx", header=None) # Le o arquivo do excel que contém a matriz A sem cabeçalho
    df_vetor = pd.read_excel(r"dados/vetor_b.xlsx", header=None) # Le o arquivo do excel que contém o vetor b sem cabeçalho
    matriz = np.array(df_matriz, dtype='f8') # Converte a matriz A em um array da numpy
    vetor = np.ravel(np.array(df_vetor, dtype='f8')) # Converte o vetor b em um array da numpy
    return matriz, vetor

### 1) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e avalie:
**a)** se A é tridiagonal;

In [ ]:
def verifica_tridiagonal(matriz_A):
    linhas, colunas = np.shape(matriz_A)

    for i in range(linhas):
        for j in range(colunas):
            if np.abs(i-j) <= 1 and matriz_A[i,j] == 0:
                return False
            if np.abs(i-j) >= 2 and matriz_A[i,j] != 0:
                return False
    return True

In [ ]:
matriz, vetor = ler()
verifica_tridiagonal(matriz)

**b)** se A é triangular superior;

In [ ]:
def verifica_tri_superior(matriz_A):
    linhas = len(matriz_A)

    for i in range(linhas):
        for j in range(i):
            if matriz_A[i,j] != 0:
                return False
    return True

In [ ]:
matriz, vetor = ler()
verifica_tri_superior(matriz)

**c)** se A é triangular inferior;

In [ ]:
def verifica_tri_inferior(matriz_A):
    colunas = len(matriz_A[0])

    for j in range(colunas):
        for i in range(j):
            if matriz_A[i,j] != 0:
                return False
    return True

In [ ]:
matriz, vetor = ler()
verifica_tri_inferior(matriz)

**d)** se Traço de A é maior que 4.

In [ ]:
def traco(matriz_A):
    linhas, colunas = np.shape(matriz_A)
    n, traco = min((linhas, colunas)), 0

    for i in range(n):
        traco += matriz_A[i,i]
    if traco > 4:
        print("O traço da matriz é " + str(traco) + ", portanto é maior que 4")
        return True
    print("O traço da matriz é " + str(traco) + ", portanto não é maior que 4")
    return False

In [ ]:
matriz, vetor = ler()
traco(matriz)

---
### 2) Implemente um algoritmo que leia uma matriz A (triangular) e um vetor b (de uma planilha excel) e resolva o sistema Ax = b por substituição para frente ou para trás conforme o caso.

In [ ]:
matriz, vetor = ler()
delta_x = resolve_substituicao(matriz, vetor)

print("Vetor x resultado da substituição:\n",delta_x,"\n")

---
### 3) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e:

### a) Avalie se é possível obter o fator de Cholesky;

### b) Resolva o sistema Ax = b usando a fatoração de Cholesky.

In [ ]:
# TESTE DO ALGORITMO DE DECOMPOSIÇÃO DE CHOLESKY E NORMA RESIDUAL (cholesky.py, normas.py) '''
matriz, vetor = ler()
delta_x = decomposicao_cholesky(matriz, vetor)

print("Vetor x resultado da Decomposição de Cholesky:\n",delta_x,"\n")
#normas.residual(matriz, vetor, delta_x)

---
### 4) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel) e:

### a) Avalie se algum critério é satisfeito para aplicação dos métodos de Gauss-Seidel, Jacobi e SOR;

### b) Resolva o sistema Ax = b usando os métodos iterativos de Gauss-Seidel, Jacobi e SOR.

---
### 5) Implemente um algoritmo que leia uma matriz A (de uma planilha excel) e:

### a) Calcule a transposta de A;

### b) Calcule o determinante de A;

### c) Calcule a matriz adjunta de A;

### d) Calcule o traço de A;

### e) Avalie se A é simétrica;

### f) Avalie se A é ortogonal;

### g) Calcule a norma de linha, norma de Frobenius e norma coluna de A.

### h) Avalie se A é positiva definida.

---
### 6) Construa um algoritmo que receba dois vetores x e y e uma matriz A positiva definida (de uma planilha excel). Calcule:

### a) Norma 1, norma 2 e norma infinito de x e y.

### b) Ângulo entre x e y;

### c) Norma de x e y induzida por A;

### d) Produto interno de x e y.

---
### 7) Implemente um algoritmo que leia uma matriz A e um vetor b (de uma planilha excel). Encontre a fatoração LU de A.

In [ ]:
# TESTE DO ALGORITMO DE FATORAÇÃO LU E NORMA RESIDUAL (fatoracaoLU.py, normas.py)'''
delta_x = fatoracaoLU(matriz,vetor)

print("Vetor x resultado da Fatoração LU:\n",delta_x,"\n")
#normas.residual(matriz, vetor, delta_x)

---
### 8) Construa um algoritmo que receba uma matriz A positiva (de uma planilha excel). Calcule:

### a) O número condição;

### b) Retorne uma mensagem ao usuário sobre sua interpretação.

---
### 9) Construa um algoritmo que receba três matrizes: U, sigma e V (de uma planilha excel) e:

### a) Avalie se o produto das três matrizes representa uma decomposição SVD de alguma matriz X;

### b) Caso a resposta à pergunta anterior seja negativa, retorne ao usuário uma explicação justificando esse fato.

---
### 10) Construa um algoritmo que leia uma planilha em Excel com 4 colunas e calcule a estatística de Benford (para 1 dígito) para cada uma das colunas.